In [ ]:
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
import pandas as pd

# NLTK 데이터 다운로드 (최초 실행 시 필요)
nltk.download('stopwords')

# 한국어 불용어 리스트 초기화 (리스트로 시작)
korean_stopwords = list(stopwords.words('english'))  # 영어 불용어를 기본으로 사용

# 한국어 불용어 추가 (필요 시 확장 가능)
korean_stopwords.extend([
    "것", "이", "을", "를", "은", "는", "에", "의", "로", "으로", "에서", "하고", "보다", "다음", "설명",
    "틀린", "옳은", "중", "해당", "사용되는", "기존", "설명으로", "내용으로만", "관련한", "관한", "구조가",
    "구조의", "정의한", "정의", "위한", "되는", "된다", "되는가", "하나로", "처리하기", "위해", "제시된",
    "되어야", "제시하는", "보기를", "제시된", "보기", "해당하는", "있는", "없는", "아닌", "정답은", "조건",
    "설정", "해석", "기반으로", "출력되는", "가장", "것은", "설계기법", "바르게", "설명하는", "기능을",
    "수행하는", "용어는", "내용은", "옳지", "않은"
])

# 추가된 영어 불용어 리스트 (기술 키워드와 관련 없는 단어)
additional_english_stopwords = [
    "the", "and", "of", "in", "for", "is", "on", "with", "this", "that", "to", "a", "an", "as", "by", 
    "it", "from", "at", "are", "use", "data", "model", "use", "you", "which", "be", "will", "have", "has"
]

# 기존 영어 불용어 리스트와 합치기
korean_stopwords.extend(additional_english_stopwords)

# 기술 키워드 사전 (확장형)
tech_keywords = set([
    "소프트웨어", "요구사항", "설계", "모델링", "프로세스", "애자일", "스크럼", "객체지향", "상속", "캡슐화", "다형성",
    "클래스", "속성", "모듈", "컴포넌트", "추상화", "정규화", "트랜잭션", "인덱스", "뷰", "릴레이션", "SQL", "DML", "DDL",
    "운영체제", "스레드", "스케줄링", "페이징", "메모리", "파일시스템", "네트워크", "TCP", "UDP", "IP", "HTTP", "라우팅",
    "프로토콜", "단위테스트", "통합테스트", "화이트박스", "블랙박스", "보안", "암호화", "인증", "C언어", "JAVA", "Python",
    "포인터", "배열", "함수", "라이브러리", "문자열", "컴파일", "응집도", "결합도", "프레임워크", "IDE", "MVC", "REST", "API",
    "UI", "UX", "인터페이스", "모니터링", "가용성", "신뢰성", "유지보수", "트리", "그래프", "정렬", "검색", "스택", "큐", "연결리스트",
    "패킷", "라우터", "세션", "토큰", "디버깅", "디자인패턴", "도메인", "식별자", "컨테이너", "정책", "백업", "복구", "UML",
    "데이터베이스", "정규형", "SQL문", "프로그래밍", "버전관리", "리눅스", "가상화", "IP주소", "디지털서명", "로그파일",
    "포인터", "메모리관리", "TLS", "SSL", "XSS", "CSRF", "DDoS", "클라우드", "VPN", "시큐어코딩"
])

# JSON 파일 로드
with open('./2022년 1회 정처기 필기.json', encoding='utf-8') as f:
    data = json.load(f)

# 질문과 선택지에서 텍스트 데이터 추출
def extract_text(data):
    text_data = []
    for item in data:
        question = item.get("question", "")
        choices = [item.get(f"choice_{i}", "") for i in range(1, 5)]
        text_data.append(question)
        text_data.extend(choices)
    return text_data

# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    text = re.sub(r'\d+', '', text)      # 숫자 제거
    # 이미지 데이터나 URL 제거 (강력하게 처리)
    text = re.sub(r'http\S+|www\S+', '', text)  # URL 제거
    text = re.sub(r'data:image/\w+;base64,[A-Za-z0-9+/=]+', '', text)  # base64 이미지 데이터 제거
    # 불필요한 문자들 제거 (의미 없는 연속된 문자, 특수문자 등)
    text = re.sub(r'[^가-힣a-zA-Z\s]', '', text)  # 한글, 영어, 공백만 남기기
    return text

# 주요 키워드 추출 함수 (각 문제별로)
def extract_keywords_for_question(text_data, top_n=5):
    preprocessed_texts = [preprocess_text(text) for text in text_data]
    vectorizer = TfidfVectorizer(stop_words=korean_stopwords, max_features=5000)
    tfidf_matrix = vectorizer.fit_transform(preprocessed_texts)
    feature_names = vectorizer.get_feature_names_out()
    
    # TF-IDF 점수 합산 후 상위 키워드 추출
    scores = tfidf_matrix.sum(axis=0).A1
    keywords = [(feature_names[i], scores[i]) for i in range(len(feature_names))]
    keywords = sorted(keywords, key=lambda x: x[1], reverse=True)[:top_n]

    # 잘못된 키워드 제거: URL, 이미지 데이터, 기타 비정상적인 텍스트
    filtered_keywords = [keyword for keyword, _ in keywords if not re.match(r'http\S+|www\S+|data:image/', keyword)]
    
    return filtered_keywords

# 전체 키워드 추출 (TF-IDF 기반)
all_keywords = []
for idx, item in enumerate(data):
    # 각 문제와 선택지 텍스트 결합
    text_data = extract_text([item])
    # 해당 문제의 상위 키워드 추출
    top_keywords = extract_keywords_for_question(text_data)
    all_keywords.append({
        "문제 번호": idx + 1,
        "TF-IDF 키워드": top_keywords
    })

# Step 1: 빈칸 문제 번호들 자동 추출 (기술 키워드가 결측치인 문제 번호 추출)
empty_problem_numbers = [item["문제 번호"] for item in all_keywords if not item["TF-IDF 키워드"]]

# Step 2: 빈칸 문제 번호에 대해 TF-IDF 키워드를 채워넣기
for item in all_keywords:
    if item["문제 번호"] in empty_problem_numbers:
        item["TF-IDF 키워드"] = ["기본 키워드"]  # 수동으로 기본 키워드 추가

# 결과를 DataFrame으로 변환하여 저장
df = pd.DataFrame(all_keywords)

# CSV 파일로 저장
df.to_csv("2022년 1회.csv", index=False, encoding="utf-8-sig")

# JSON 파일로 저장
with open("2022년 1회.json", "w", encoding="utf-8") as f:
    json.dump(all_keywords, f, ensure_ascii=False, indent=2)

df.head()  # 결과 확인
